In [1]:
# This needs to be rewritten. Currently clips are taken and analyzed in 2 second chunks. 
# Audio needs to come in as a stream. Every X number of milliseconds OR after the FFT/Prediction is completed, the next
# 2048 samples need to be taken from the stream and analyzed. 

# https://forum.pjrc.com/threads/24492-Using-the-PDB-on-Teensy-3#post_message_37659
# https://forum.pjrc.com/threads/24611-Problem-with-DMA-interrupt
# https://forum.pjrc.com/threads/18237-teensy-3-0-memory-to-memory-DMA-help

# https://python-sounddevice.readthedocs.io/en/0.3.11/examples.html

In [2]:
import tinn
import random
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import Queue
from collections import deque
from threading import Timer
from threading import Thread
from threading import Event
import time

In [3]:
#https://pymotw.com/2/threading/

In [4]:
class Data:
    def __init__(self, path, nips, nops):
        self.read_data(path, nips, nops)

    def __repr__(self):
        return str( str(self) +'rows with'+ str(len(self.in_[0])) +'inputs and'+ str(len(self.tg[0])) +'outputs' )
        #return f'{len(self)} rows with {len(self.in_[0])} inputs and {len(self.tg[0])} outputs.'

    def read_data(self, path, nips, nops):
        self.in_, self.tg = [], []
        with open(path) as data_file:
            for line in data_file:
                row = list(map(float, line.split()))
                self.in_.append(row[:nips])
                self.tg.append(row[nips:])

    def shuffle(self):
        indexes = list(range(len(self.in_)))
        random.shuffle(indexes)
        self.in_ = [self.in_[i] for i in indexes]
        self.tg = [self.tg[i] for i in indexes]

    def __len__(self):
        return len(self.in_)
    
    def dLen(self):
        return len(self.in_)

In [5]:
class DataOnTheFly:
    def __init__(self, arr, nips, nops):
        self.read_array(arr, nips, nops)
        
    def read_array(self, arr, nips, nops):
        self.in_, self.tg = [], []

        self.in_.append(arr[:nips])
        self.tg.append(arr[nips:])
        

In [6]:
def train(_nips, _nhid, _nops, _rate, _anneal, _data, _iter):
    nips = _nips
    nhid = _nhid
    nops = _nops
    rate = _rate
    anneal = _anneal
    
    data = Data(_data, nips, nops)
    
    t = tinn.Tinn(nips, nhid, nops)
    
    for i in range(_iter):
        data.shuffle()
        error = 0
        for in_, tg in zip(data.in_, data.tg):
            error += tinn.xttrain(t, in_, tg, rate)
        print 'Error: ' + str(error/len(data)) + ' learning rate: ' + str(rate) + ' Iteration: ' + str(i)
        rate *= anneal

    t.save('savedApp.tinn')

In [7]:
def predict(arr, _nips, _nops):

    data = DataOnTheFly(arr, _nips, _nops)
    
    loaded = tinn.xtload('savedApp.tinn')

    in_ = data.in_[0]
    tg = data.tg[0]
        
    pd = tinn.xtpredict(loaded, in_)
        
    #print(' '.join(map(str, tg)))
    #print(' '.join(map(str, pd)))
    
    return pd

In [8]:
#https://stackoverflow.com/questions/12435211/python-threading-timer-repeat-function-every-n-seconds
#Repeated Timer
class MyThread(Thread):
    def __init__(self, event):
        Thread.__init__(self)
        self.stopped = event

    def run(self):
        while not self.stopped.wait(0.0001):
            #Call a function
            addToAudioBuffer()

In [9]:
def addToAudioBuffer():

    #if transferBuff isn't empty
    if len(transferBuff) > 1:

        #Get transferBuff - Channel 0
        aClip = list(transferBuff)[0]

        #Clear it
        transferBuff.clear()

        #Unpack data and push to audioBuffer
        for i in range(len(aClip)):
            audioBuffer.append(aClip[i][0])


def callback(indata, frames, time, status):
    #print 'callback', frames, len(indata), type(indata), indata
    if status:
        print(status)
        
    #Add indata to queue
    transferBuff.append(indata)


In [10]:
def getFFT(_aClip):
    
    fs = 1024
    duration = 2
    
    #Get data
    #data = aClips[i]
    data = _aClip

    #Smooth the FFT by windowing data
    data = data * np.hanning(len(data))

    #Preform FFT and take only real data
    fft = abs(np.fft.fft(data).real)

    #Keep only first half
    fft = fft[:int(len(fft))/2]

    #Create Frequency list
    freq = np.fft.fftfreq( (fs * duration) ,1.0/fs)

    #Keep only first half
    freq = freq[:int(len(freq))/2] 

    #At this point, the length of the fft/freq is cut in reduced from 2048 to 1024. But the cut off frequency is actually 512.
    #The freq axis shows why. Values are now 0, 0.5, 1, 1.5Hz...
    #Therefore drop every other value
    fft = fft[0::2]
    freq = freq[0::2]

    #Save for later
    #ffts.append( fft )
    
    return fft

In [11]:
def formatDataForPrediction(arr):
    #Convert to list
    arr = arr.tolist()
    
    #Append 1 0 to list - woof
    arr = arr + [1.0, 0.0]
    
    return arr

In [12]:
#Train NN
#train(512, 32, 2, 0.5, 0.99, 'woofsAndMeows.data', 32)

In [13]:
#Audio Buffer 
audioBuffer = deque( maxlen=2048 )

#Buffer used to push data from callback to audioBuffer
transferBuff = deque()

#Create events
stopFlag = Event()

#Create threads
thread = MyThread(stopFlag)
thread.start()

In [27]:
#Define Run time in seconds
duration = 2000

#Clear flag
stopFlag.clear()

with sd.InputStream(channels=1, callback=callback, samplerate=1024):
    print 'Buffering...'
    time.sleep(2)
    
    count = 0
    print 'Waiting for woof'
    while(count < duration):
        
        aClip = list(audioBuffer)
        
        if len(aClip) != 2048:
            print len(aClip)
            continue
        
        else:

            #print count, len(audioBuffer), type(audioBuffer), audioBuffer[0]
            count = count + 1

            clipFFT = getFFT( aClip )

            if len(clipFFT) != 512:
                print 'FFT Size error'

            #Format for NN
            formattedClip = formatDataForPrediction(clipFFT)

            ##Predict
            predictedValues = predict( formattedClip, 512, 2 )
            #print predictedValues[0], predictedValues[1], aClip[0], aClip[2047]

            #Found woof
            if predictedValues[0] >= 0.76 and predictedValues[1] <= 0.25:
                print 'Found Woof'
                print predictedValues
            #Found meow   
            elif predictedValues[0] > 0.4 and predictedValues[1] > 0.7:
                print 'Meow'

            time.sleep(0.01)

print 'Done'

sd.stop()
stopFlag.set()
#thread.join()

Buffering...
Waiting for woof
Found Woof
[0.7734981227175097, 0.22506987955441338]
Found Woof
[0.7883847441097479, 0.2103362220386945]
Found Woof
[0.7871614287336235, 0.2116297617802646]
Found Woof
[0.8109919735289326, 0.1880087259616406]
Found Woof
[0.7898779029625416, 0.2092603090860193]
Found Woof
[0.7671403344876534, 0.23202665578348974]
Found Woof
[0.7865391246767623, 0.21260125783632813]
Found Woof
[0.7815088560743636, 0.2175925843931817]
Found Woof
[0.7853504096363111, 0.21363343061113352]
Found Woof
[0.7792713390331086, 0.21952569845392286]
Found Woof
[0.7919441727221419, 0.2067605844475197]
Found Woof
[0.7748438226544198, 0.22357809752899868]
input overflow
Found Woof
[0.7670051094447018, 0.23099930892785764]
Found Woof
[0.7785037529946098, 0.2197187544964406]
Found Woof
[0.7722334650369356, 0.22683044366651012]
input overflow


KeyboardInterrupt: 